# DEA - Surface Reflectance Intercomparison Tool

{{ add blurb }}

In [1]:
# Requirements
import os
import os.path
import sys
import importlib.util

def load_dea_script(module_name, script_dir=None):
    """ This function enables importing files from other collections in dea-notebooks """
    script_dir = script_dir or os.path.dirname(os.getcwd()) + '/10_Scripts'
    spec = importlib.util.spec_from_file_location(module_name, script_dir + '/' + module_name + '.py')
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    sys.modules[module_name] = module

load_dea_script('DEADataHandling')
import DEADataHandling
    
from datacube import Datacube
from datacube.config import LocalConfig
from configparser import ConfigParser

from ipywidgets import Dropdown, DatePicker
import ipywidgets as widgets

In [2]:
"""
Datacube UI functions
"""
def get_datacube_environments(cfg_path=None):
    """ Reads datacube environments from default configuration paths"""
    if cfg_path:
        lc = LocalConfig(cfg_path)
    else:
        lc = LocalConfig.find()
    for fn in lc.files_loaded:
        _cfg = ConfigParser()
        _cfg.read(fn)
        for _section in _cfg.sections():
            yield (_section, fn)
            
def get_datacube_products(dc_env, dc_cfg):
    """ Returns products defined for a given datacube """
    dc = Datacube(config=dc_cfg, env=dc_env)
    for index, row in dc.list_products().iterrows():
        yield index, row['name']

class DCProduct:
    def __init__(self):
        self._cfg = widgets.Dropdown(
            description="cfg path:",
                options=[
                    (
                        "{} -- {}".format(env, filen),
                        "{}:{}".format(env, filen)
                    )
                    for (env, filen) in get_datacube_environments()
                ]
        )
        
        self._product = widgets.Dropdown(
            description="product:",
            options=[]
        )
        
        self._bands = widgets.SelectMultiple(
            description="bands:",
            options=[]
        )
        self._cfg.observe(self._update_product_options, names='value')
        self._product.observe(self._update_band_options, names='value')
        self._update_product_options({'new': self._cfg.options[0][1]})
        
    def _update_product_options(self, change):
        self._product.options = [
            (product, p_id) for (p_id, product) in get_datacube_products(
            *change['new'].split(':'))
        ]
        
    def _update_band_options(self, change):
        self._bands.options = list(
            self.datacube
            .index.products
            .get(change['new']).measurements
        )
        
    def generate_display(self):        
        return widgets.VBox([
            self._cfg,
            self._product,
            self._bands
        ])
    
    @property
    def datacube(self):
        dc_env, dc_cfg = self._cfg.value.split(':')
        return Datacube(config=dc_cfg, env=dc_env)
    
    @property
    def product_id(self):
        return self._product.value
    
    @property
    def selected_bands(self):
        return self._bands.value

    
class DCProductSelector:
    """
    Class wrapper to configure the products for the experiment
    Generates a UI to select the datacube configuration, product 
        and bands for product to compare
    """
    
    def __init__(self, product_count=4):
        self.product_count = product_count
        self._datacube_products = []
        for i in range(product_count):
            self._datacube_products.append(DCProduct())
            
    def generate_display(self):
        tab_nest = widgets.Tab()
        for idx, dcp in enumerate(self._datacube_products):
            title = 'Product %d' % (idx + 1)
            tab_nest.set_title(idx, title)
            
        tab_nest.children = tuple(_dcp.generate_display() for _dcp in self._datacube_products)
        
        return tab_nest

    
class StudyAreaSelector:
    
    def __init__(self):
        self._start = DatePicker(
            description='Start date:',
            disabled=False
        )
        self._end = DatePicker(
            description='End date:',
            disabled=False
        )
        
    @property
    def start(self):
        return getattr(self._start, 'value', None)
    
    @property
    def end(self):
        end = getattr(self._end, 'value', None)
        start = self.start
        if start and start > end:
            raise ValueError("Start date is configured after end date")
        return end
    
    def generate_display(self):
        return widgets.VBox([
            self._start,
            self._end
        ])


study_area = StudyAreaSelector()
product_selector = DCProductSelector()

### Product Configuration

Select the products you would like to compare from Datacube

In [3]:
display(product_selector.generate_display())

### Area of Study Configuration

Select your study area

In [4]:
display(study_area.generate_display())

In [5]:
for dc in product_selector._datacube_products:
    print(dc.product_id, dc.selected_bands, len(dc.datacube.find_datasets(product='bom_rainfall_grids')))

26 () 41591
36 () 41591
36 () 41591
36 () 41591
